In [14]:
import pandas as pd
import seaborn as sns
df = sns.load_dataset("penguins")

In [17]:
df[df.species=='Gentoo']

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
220,Gentoo,Biscoe,46.1,13.2,211.0,4500.0,Female
221,Gentoo,Biscoe,50.0,16.3,230.0,5700.0,Male
222,Gentoo,Biscoe,48.7,14.1,210.0,4450.0,Female
223,Gentoo,Biscoe,50.0,15.2,218.0,5700.0,Male
224,Gentoo,Biscoe,47.6,14.5,215.0,5400.0,Male
...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female


In [5]:
df.species.unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [7]:
df.sex.unique()

array(['Male', 'Female', nan], dtype=object)

In [8]:
df.describe()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,344.000000,342.000000,342.000000,342.000000,342.000000
mean,0.918605,43.921930,17.151170,200.915205,4201.754386
std,0.893320,5.459584,1.974793,14.061714,801.954536
min,0.000000,32.100000,13.100000,172.000000,2700.000000
25%,0.000000,39.225000,15.600000,190.000000,3550.000000
50%,1.000000,44.450000,17.300000,197.000000,4050.000000
75%,2.000000,48.500000,18.700000,213.000000,4750.000000
max,2.000000,59.600000,21.500000,231.000000,6300.000000


In [8]:
from sklearn.preprocessing import LabelEncoder
encoder1=LabelEncoder()
df['species'] = encoder1.fit_transform(df['species'])
df.species.unique()

array([0, 1, 2], dtype=int64)

In [9]:
X = df.drop(columns=['species'])
y = df['species']

In [10]:
X.head()

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Torgersen,36.7,19.3,193.0,3450.0,Female


In [11]:
y.head()

					sex

0    0
1    0
2    0
3    0
4    0
Name: species, dtype: int64

In [5]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# cargar dataset de pinguinos
df = sns.load_dataset("penguins")

# codificar variable objetivo
encoder1=LabelEncoder()
df['species'] = encoder1.fit_transform(df['species'])

# división en variables características (X) y variable objetivo (y)
X = df.drop(columns=['species'])
y = df['species']

# división del en X_train, y_train, X_test e y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# imputación de columnas categóricas
categorical_columns = ['island','sex']
cat_imputer = SimpleImputer(strategy='most_frequent')
X_train.loc[:, categorical_columns] = cat_imputer.fit_transform(X_train[categorical_columns])
X_test.loc[:, categorical_columns] = cat_imputer.transform(X_test[categorical_columns])

# imputación de columnas numéricas
numerical_columns = ['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']
num_imputer = SimpleImputer(strategy='mean')
X_train.loc[:, numerical_columns] = num_imputer.fit_transform(X_train[numerical_columns])
X_test.loc[:, numerical_columns] = num_imputer.transform(X_test[numerical_columns])


variables_numericas = X_train.select_dtypes(include=['float64','int64']).columns
scaler = StandardScaler()
X_train[variables_numericas] = scaler.fit_transform(X_train[variables_numericas])
X_test[variables_numericas] = scaler.transform(X_test[variables_numericas])


variables_categoricas = X_train.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False, drop = 'first')
X_train_encoded = encoder.fit_transform(X_train[variables_categoricas])
X_test_encoded = encoder.transform(X_test[variables_categoricas])


X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(variables_categoricas))
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(variables_categoricas))

X_train_prepared = pd.concat([X_train[variables_numericas].reset_index(drop=True), X_train_encoded_df.reset_index(drop=True)], axis=1)
X_test_prepared =  pd.concat([X_test[variables_numericas].reset_index(drop=True), X_test_encoded_df.reset_index(drop=True)], axis=1)


from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(
        n_estimators = 100,            # Establece el número de árboles en el bosque a 100.
           max_depth = 5,              # Limita la profundidad máxima de cada árbol a 5 para evitar el sobreajuste.
        class_weight = 'balanced',     # Ajusta los pesos de las clases para manejar el desbalance en los datos; 
                                       # las clases menos frecuentes recibirán más peso.
        random_state = 42              # Establece una semilla aleatoria para asegurar la reproducibilidad del modelo.
)

random_forest.fit(X_train_prepared, y_train)


import pickle

# Guardar el modelo RandomForest
pickle.dump(random_forest, open('rf_pinguinos.pkl', 'wb'))

# Guardar el OneHotEncoder
pickle.dump(encoder, open('encoder_pinguinos.pkl', 'wb'))

# Guardar el StandardScaler
pickle.dump(scaler, open('scaler_pinguinos.pkl', 'wb'))

# Guardar el DataFrame `X_train_prepared` completo para referencia en predicciones
pickle.dump(X_train_prepared, open('X_train_prepared_pinguinos.pkl', 'wb'))




Nombres de las columnas originales esperadas:
['island' 'sex']

Nombres de las columnas en categorical_features:
Index(['island', 'sex'], dtype='object')


In [9]:
print(input_df.columns)

Index(['island', 'bill_length_mm', 'bill_depth_mm', 'flipper_length_mm',
       'body_mass_g', 'sex'],
      dtype='object')


In [13]:
X_train_prepared

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_Dream,island_Torgersen,sex_Male
0,-2.005109,-0.534410,-1.645021,-1.636340,1.0,0.0,0.0
1,-0.816790,1.761560,-0.708965,-0.388429,0.0,0.0,0.0
2,-1.529781,-0.085198,-0.852974,-1.074780,0.0,1.0,0.0
3,1.084521,-0.634235,1.667176,1.421042,0.0,0.0,1.0
4,1.066240,-0.534410,0.875129,1.483437,0.0,0.0,1.0
...,...,...,...,...,...,...,...
270,-1.145863,-0.035286,-1.068987,-1.137176,0.0,1.0,1.0
271,0.371530,-1.033534,1.019137,1.109064,0.0,0.0,1.0
272,-0.231771,0.663487,-0.420948,0.048339,0.0,1.0,1.0
273,-1.346963,0.813224,-0.996982,-0.513221,0.0,1.0,0.0


In [14]:
X_test_prepared

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_Dream,island_Torgersen,sex_Male
0,-0.469435,0.663487,0.011078,-0.263638,1.0,0.0,1.0
1,-0.835071,0.114451,-1.068987,-0.513221,0.0,1.0,0.0
2,0.426375,-1.083446,1.091142,0.609899,0.0,0.0,0.0
3,0.865139,-0.484497,1.523167,2.232184,0.0,0.0,1.0
4,0.371530,0.513750,-1.645021,-1.199572,1.0,0.0,0.0
...,...,...,...,...,...,...,...
64,0.536066,0.064539,-1.140991,-0.638012,1.0,0.0,0.0
65,-0.378026,-1.832132,0.659116,-0.076452,0.0,0.0,0.0
66,-1.657754,0.364013,-0.780969,-0.949989,0.0,0.0,0.0
67,-1.164145,1.312348,-0.204935,-0.887594,0.0,1.0,1.0


In [15]:
num_imputer

SimpleImputer()

In [17]:
load_clf = pickle.load(open('rf_pinguinos.pkl', 'rb'))
encoder = pickle.load(open('encoder_pinguinos.pkl', 'rb'))
scaler = pickle.load(open('scaler_pinguinos.pkl', 'rb'))
X_train_prepared = pickle.load(open('X_train_prepared_pinguinos.pkl', 'rb'))  # DataFrame de referencia

In [3]:
input_df=pd.read_excel('prueba.xlsx')
input_df

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Torgersen,39.1,18.7,181,375,Male


In [4]:
# Codificación OneHot con el codificador cargado
categorical_features = input_df[['island','sex']]
categorical_features = categorical_features[['island','sex']]
categorical_features

,island,sex
0,Torgersen,Male


In [21]:
encoded_features = encoder.transform(categorical_features)

In [22]:
# Codificación OneHot con el codificador cargado
categorical_features = input_df[['island','sex']]
encoded_features = encoder.transform(categorical_features)
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['island','sex']))

# Escalar características numéricas con el escalador cargado
numerical_features = input_df[['bill_length_mm','bill_depth_mm','flipper_length_mm','body_mass_g']]
scaled_numerical = scaler.transform(numerical_features)
scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=numerical_features.columns)

# Concatenar datos procesados
input_df_processed = pd.concat([scaled_numerical_df, encoded_df], axis=1)

# Alinear y rellenar `input_df_processed` para que coincida con `X_train_prepared`
input_df_processed = input_df_processed.reindex(columns=X_train_prepared.columns, fill_value=0)